In [169]:
import numpy as np

def clock_pulse(t,period=1):
    return np.sin(period*t) > 0

class FlipFlopD():
    def __init__(self, Q=True):
        self.latch = LatchSR(Q)
        self.Q = self.latch.Q
        self.notQ = self.latch.notQ
        
    def run(self, D, t=0):
        clk = clock_pulse(t)
        #primer bloque
        R = not(D and clk)
        S = not(not(D) and clk)
        
        self.latch.run(S,R)
        self.Q = self.latch.Q
        self.notQ = self.latch.notQ

In [170]:
flipflop = FlipFlopD()

for pulse in [0, 1]:
    for d in [False, True]:
        flipflop.run(d, t=pulse)
        print(clock_pulse(pulse), d, flipflop.Q)

False False False
False True False
True False False
True True True


In [171]:
flipflop.run(True, t=1)
flipflop.Q

True

# Flip-Flop T

In [57]:
class LatchSR():
    def __init__(self, Q=True):
        self.Q = Q
        self.notQ = not(Q)
    
    def run(self, S, R):
        if S == False and R == False:
            self.Q, self.notQ = self.Q, self.notQ
        elif S == True:
            self.notQ = not(S or self.Q)
            self.Q = not(R or self.notQ)
        elif S == False:
            self.Q = not(R or self.notQ)
            self.notQ = not(S or self.Q)

In [58]:
for s in [False, True]:
    for r in [False, True]:
        sr = LatchSR()
        sr.run(s, r)
        print(s,r, sr.Q, sr.notQ)

False False True False
False True False True
True False True False
True True False False


In [59]:
class FlipFlopT():
    def __init__(self, Q=True):
        self.latch = LatchSR(Q)
        self.Q = self.latch.Q
        self.notQ = self.latch.notQ
        
    def run(self, T, clk=True, t=0):
        #clk = clock_pulse(t)
        #primer bloque
        R = T and clk and self.Q
        S = T and clk and self.notQ
        #Segundo bloque
        self.latch.run(S,R)
        self.Q = self.latch.Q
        self.notQ = self.latch.notQ

In [60]:
ffT = FlipFlopT(Q=False)
print(ffT.Q, ffT.notQ, end=' \\ ')
ffT.run(T=True, clk=False)
print(ffT.Q, ffT.notQ)

False True \ False True


In [61]:
ffT.run(T=True)
print(ffT.Q, ffT.notQ)

True False


In [62]:
class counter3():
    def __init__(self):
        self.module0 = FlipFlopT(Q=False)
        self.module1 = FlipFlopT(Q=False)
    
    def count(self, clk=False):
        self.module0.run(T=True, clk=clk)
        self.module1.run(T=self.module0.Q, clk=clk)
        
        return 1*self.module0.Q + 2*1*self.module1.Q

In [63]:
counter = counter3()

In [64]:
for t in range(0,4):
    print(counter.count(clk=True))

3
2
1
0


In [65]:
class counter7():
    def __init__(self):
        self.module0 = FlipFlopT(Q=False)
        self.module1 = FlipFlopT(Q=False)
        self.module2 = FlipFlopT(Q=False)
        self.module3 = FlipFlopT(Q=False)
    
    def count(self, clk=False):
        self.module0.run(T=True, clk=clk)
        self.module1.run(T=self.module0.Q, clk=clk)
        input_module2 = self.module0.Q and self.module1.Q
        self.module2.run(T=input_module2, clk=clk)
        
        return 1*self.module0.Q + 2*1*self.module1.Q + 4*1*self.module2.Q

In [66]:
counter = counter7()
for t in range(0,8):
    print(counter.count(clk=True))

7
6
5
4
3
2
1
0


In [73]:
class DecimalCounter():
    def __init__(self):
        self.module0 = FlipFlopT(Q=False)
        self.module1 = FlipFlopT(Q=False)
        self.module2 = FlipFlopT(Q=False)
        self.module3 = FlipFlopT(Q=False)
    
    def count(self, clk=False):
        self.module0.run(T=True, clk=clk)
        
        input_module1 = self.module0.Q and self.module3.notQ
        self.module1.run(T=self.module0.Q and self.module3.notQ, clk=clk)
        
        input_module2 = self.module0.Q and self.module1.Q
        self.module2.run(T=self.module0.Q and self.module1.Q, clk=clk)
        
        Q0andQ1andQ2 = input_module2 or self.module2.Q
        Q0andQ3 = self.module0.Q and self.module3.Q
        input_module3 = Q0andQ1andQ2 or Q0andQ3
        self.module3.run(T=input_module3, clk=clk)

        #return 1*self.module0.Q, 1*self.module1.Q, 1*self.module2.Q, 1*self.module2.Q 
        return 1*self.module0.Q + 2*1*self.module1.Q + 4*1*self.module2.Q + 8*1*self.module3.Q

In [74]:
decimal = DecimalCounter()
for t in range(0,10):
    print(decimal.count(clk=True))

15
6
13
4
11
10
7
14
3
2


In [72]:
print(decimal.count(clk=True))

14
